In [ ]:
using AcausalNets
using QI
using LightGraphs
using LinearAlgebra

In [ ]:
roA = Diagonal([.5, .5])
roBwA = Diagonal([.5, .5, .4, .6])
roCwA = Diagonal([.7, .3, .2, .8])
roDwB = Diagonal([.9, .1, .5, .5])
roEwC = Diagonal([.3, .7, .6, .4])
roFwDE = Diagonal([.01, .99, .01, .99, .01, .99, .99, .01])
roGwC = Diagonal([.8, .2, .1, .9])
roHwEG = Diagonal([.05, .95, .95, .05, .95, .05, .95, .05])

var_a = Variable(:a, 2)
var_b = Variable(:b, 2)
var_c = Variable(:c, 2)
var_d = Variable(:d, 2)
var_e = Variable(:e, 2)
var_f = Variable(:f, 2)
var_g = Variable(:g, 2)
var_h = Variable(:h, 2)

sys_a = DiscreteQuantumSystem([var_a], roA)
sys_b = DiscreteQuantumSystem([var_a], [var_b], roBwA)
sys_c = DiscreteQuantumSystem([var_a], [var_c], roCwA)
sys_d = DiscreteQuantumSystem([var_b], [var_d], roDwB)
sys_e = DiscreteQuantumSystem([var_c], [var_e], roEwC)
sys_f = DiscreteQuantumSystem([var_d, var_e], [var_f], roFwDE)
sys_g = DiscreteQuantumSystem([var_c], [var_g], roGwC)
sys_h = DiscreteQuantumSystem([var_e, var_g], [var_h], roHwEG)


example_an = AcausalNet()
push!(example_an, sys_a)
push!(example_an, sys_b)
push!(example_an, sys_c)
push!(example_an, sys_d)
push!(example_an, sys_e)
push!(example_an, sys_f)
push!(example_an, sys_g)
push!(example_an, sys_h)

# gplot(example_bn.dag, nodelabel=nodes(example_bn))
# [v.target for v in example_bn.cpds]

In [ ]:
an = example_an;

In [ ]:
import AcausalNets.Inference: moral_graph
mg = moral_graph(an)
println(variables_names(an))
mg.fadjlist

In [ ]:
import AcausalNets.Inference: triangulate
triangulated, clusters = triangulate(mg, an)

println(variables_names(an))
triangulated.fadjlist

In [ ]:
import AcausalNets.Inference: JoinTree

jt = JoinTree(clusters, an);

In [ ]:
import AcausalNets.Inference: initialize

init_jt = initialize(jt, an)

In [ ]:
[[ds.variables for ds in cluster] for cluster in propagated_jt.clusters]

In [ ]:
import AcausalNets.Inference: single_message_pass

In [ ]:
# bc from bcd
ptrace(init_jt.vertex_to_num[5], [2,2,2], [3])

In [ ]:
# b from abc
ptrace(init_jt.vertex_to_num[6], [2,2,2], [1])

In [ ]:
passed = single_message_pass(5, 6, init_jt, an)
passed_2 = single_message_pass(6, 5, passed, an);

ptrace(passed_2.vertex_to_num[5], [2,2,2], [3])

In [ ]:
ptrace(passed_2.vertex_to_num[6], [2,2,2], [1])
# bc in abc and bcd are now consistent

In [ ]:
import AcausalNets.Inference: global_propagation

propagated_jt = global_propagation(init_jt, an)

In [ ]:
a = 1
a_oper = ketbra(a,a,2)
assignments = Dict([
        var_a => a_oper
        ])

initialized_jt = initialize(jt, an, assignments)
propagated_jt = global_propagation(initialized_jt, an)
propagated_jt.clusters

aBC = propagated_jt.vertex_to_num[6]
ptrace(aBC, [2,2,2], [2,3])